In [1]:
# biblioteki
from bs4 import BeautifulSoup 
import requests as req 
import time
import datetime
import pandas as pd
import numpy as np
import concurrent.futures

In [2]:
checked_range=range(1,312)

In [27]:
div_elements=[]
for i in checked_range:
    url = f"https://gratka.pl/nieruchomosci/mieszkania/wielkopolskie?page={i}&gclid=Cj0KCQjw7PCjBhDwARIsANo7CgkQq3sz7QjamydfPdvSp2zNOQe3oTABLQm-7F35qzOjrlwdUO75yoYaAitAEALw_wcB"
    response=req.get(url)
    html_content=response.text
    soup=BeautifulSoup(html_content, 'html.parser')
    div_element=soup.find_all("div", {"class":"listing__teaserWrapper"})
    div_elements.append(div_element)

In [28]:
li_list=[]
locations_list=[]
prices_list=[]
info_list=[]
for div in div_elements:
    for d in div:
        location=d.find("span",{"class":"teaserUnified__location"}).text.strip()
        locations_list.append(location)
        price=d.find("p", {"class":"teaserUnified__price"}).text
        prices_list.append(price)
        li_elements=d.find_all("li", {"class":"teaserUnified__listItem"})
        li_list.append(li_elements)

trim_locations=[]
for location in locations_list:
    trim_location=location.replace(" ", "")
    trim_locations.append(trim_location)
#trim_locations
int_whole_prices=[]
int_prices_per_m2=[]
for price in prices_list:
    trim_price=price.replace("\n","").replace("Unknown","").replace(" ","")
    whole_price=trim_price[0:trim_price.find("z")]
    int_whole_prices.append(whole_price)
    price_per_m2=trim_price[trim_price.find("ł"):].replace("ł","").replace("z/m2","")
    int_prices_per_m2.append(price_per_m2)
#print(trim_locations, int_prices)

df=pd.DataFrame(trim_locations)
df["whole_price"]=int_whole_prices
df["price_per_m2"]=int_prices_per_m2
df["info"]=li_list
for index, row in df.iterrows():
    new_info = [line.text for line in row["info"]]
    df.at[index, "info"] = new_info
expanded_info = df["info"].apply(pd.Series)
df[["size", "rooms", "floor"]] = expanded_info

In [29]:
df[['Miasto', 'Powiat', 'Województwo']]=df[0].str.split(",",expand=True)
df.drop([0,"info"],axis=1,inplace=True)

In [30]:
df

,whole_price,price_per_m2,size,rooms,floor,Miasto,Powiat,Województwo
0,708000,8000,"88,50 m2",3 pokoje,3 piętro,Koziegłowy,poznański,wielkopolskie
1,Zapytajocen,ę,"50,11 m2",2 pokoje,2 piętro,Poznań,Jeżyce,wielkopolskie
2,635000,7744,82 m2,4 pokoje,1 piętro,Czapury,poznański,wielkopolskie
3,529000,12078,"43,80 m2",2 pokoje,parter,Skórzewo,poznański,wielkopolskie
4,440000,5641,78 m2,4 pokoje,1 piętro,Luboń,Lasek,wielkopolskie
...,...,...,...,...,...,...,...,...
4845,569000,6578,"86,50 m2",5 pokoi,parter,Kórnik,poznański,wielkopolskie
4846,644244,8851,"72,79 m2",4 pokoje,4 piętro,OstrówWielkopolski,ostrowski,wielkopolskie
4847,352252,6640,"53,05 m2",2 pokoje,4 piętro,OstrówWielkopolski,ostrowski,wielkopolskie
4848,472472,6490,"72,80 m2",4 pokoje,2 piętro,OstrówWielkopolski,ostrowski,wielkopolskie


In [31]:
df.columns=["Cena_całkowita", "Cena_za_m2", "Powierzchnia","Liczba_pokoi","Piętro","Miasto","Powiat","Województwo"]
df=df[["Miasto","Powiat","Województwo","Powierzchnia","Liczba_pokoi","Piętro","Cena_za_m2","Cena_całkowita"]]

In [59]:
filt=df["Cena_całkowita"]!="Zapytajocen"
df_clean=df[filt]

In [44]:
def get_num(row):
    if type(row)==str and row=="parter":
        return 0
    elif type(row)==str:
        return row[:row.find(" ")]


In [60]:
df_clean=df_clean[(df_clean["Piętro"]!="niski")&(df_clean["Piętro"]!="poddasz")&(df_clean["Piętro"]!="powyżej")&(df_clean["Piętro"]!="poddasze")&(df_clean["Piętro"]!="niski parter")]
df_clean=df_clean[df_clean["Piętro"].notna()]
df_clean=df_clean[(df_clean["Liczba_pokoi"].notna())&(df_clean["Liczba_pokoi"]!="więcej niż 8 pokoi")]


In [46]:
df_clean["Piętro"].value_counts()

1 piętro     887
parter       739
2 piętro     563
3 piętro     485
4 piętro     315
5 piętro     106
6 piętro      62
7 piętro      33
8 piętro      25
9 piętro      22
10 piętro     19
11 piętro     17
16 piętro      9
15 piętro      8
12 piętro      6
14 piętro      5
17 piętro      4
13 piętro      4
Name: Piętro, dtype: int64

In [47]:
df_clean["Liczba_pokoi"].value_counts()

3 pokoje    1222
2 pokoje    1217
4 pokoje     506
1 pokój      259
5 pokoi       87
6 pokoi       17
7 pokoi        1
Name: Liczba_pokoi, dtype: int64

In [54]:
df_clean.loc[65, "Powierzchnia"]

'1 700 m2'

In [58]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3309 entries, 0 to 4849
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Miasto          3309 non-null   object 
 1   Powiat          3309 non-null   object 
 2   Województwo     2797 non-null   object 
 3   Powierzchnia    3309 non-null   float64
 4   Liczba_pokoi    3309 non-null   object 
 5   Piętro          2570 non-null   object 
 6   Cena_za_m2      3309 non-null   object 
 7   Cena_całkowita  3309 non-null   object 
dtypes: float64(1), object(7)
memory usage: 361.7+ KB


In [61]:
df_clean["Piętro"]=df_clean["Piętro"].apply(get_num)
df_clean["Liczba_pokoi"]=df_clean["Liczba_pokoi"].apply(get_num)
df_clean["Powierzchnia"] = df_clean["Powierzchnia"].str.replace("m2","").str.replace(",",".").str.replace(" ","").astype(float)
df_clean["Liczba_pokoi"]=df_clean["Liczba_pokoi"].astype(float)
df_clean["Piętro"]=df_clean["Piętro"].astype(float)
df_clean["Cena_za_m2"]=df_clean["Cena_za_m2"].astype(int)
df_clean["Cena_całkowita"]=df_clean["Cena_całkowita"].astype(int)

In [63]:
poznan=df_clean[df_clean["Miasto"]=="Poznań"]
poznan=poznan[poznan["Powiat"]!="wielkopolskie"]
poznan.rename({"Powiat":"Dzielnica"},axis=1,inplace=True)
poznan.drop("Województwo",axis=1,inplace=True)

In [64]:
poznan

,Miasto,Dzielnica,Powierzchnia,Liczba_pokoi,Piętro,Cena_za_m2,Cena_całkowita
5,Poznań,Jeżyce,25,1.0,0.0,13963,359000
8,Poznań,Wilda,50,2.0,2.0,10200,510000
12,Poznań,Piątkowo,58,2.0,0.0,10034,590000
13,Poznań,Łazarz,110,3.0,1.0,6984,770000
14,Poznań,StareMiasto,53,2.0,2.0,12420,664594
...,...,...,...,...,...,...,...
4811,Poznań,Strzeszyn,35,1.0,3.0,9697,340567
4812,Poznań,Strzeszyn,41,2.0,0.0,9700,402065
4813,Poznań,Strzeszyn,66,4.0,0.0,9200,611524
4814,Poznań,Strzeszyn,42,2.0,0.0,9700,412929


In [65]:
poznan_x=poznan[['Miasto', 'Dzielnica','Powierzchnia',
       'Liczba_pokoi', 'Cena_za_m2', 'Piętro']]
poznan_y=poznan[["Cena_całkowita"]]

In [66]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
num_pipeline=Pipeline([
                    ('inputer ', SimpleImputer(strategy='median')),
                    ('scaler', StandardScaler())])

num_cols=['Powierzchnia',
       'Liczba_pokoi', 'Cena_za_m2', 'Piętro']
cat_cols=['Miasto', 'Dzielnica']
full_pipeline=ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', OneHotEncoder(), cat_cols)])

In [67]:
poznan_x=full_pipeline.fit_transform(poznan_x)

In [68]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y=train_test_split(poznan_x, poznan_y, test_size=0.4)

In [69]:
poznan_x.shape

(1504, 51)

In [70]:
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_model = LinearRegression()
lin_model.fit(train_x, train_y)

# Predict on the test data
y_pred = lin_model.predict(test_x)

# Calculate the R-squared score
r_squared = lin_model.score(test_x, test_y)

# Calculate the mean squared error
mse = mean_squared_error(test_y, y_pred)

print("R-squared Score:", r_squared)
print("Mean Squared Error:", mse)

# SVR
svr_model = SVR()
svr_model.fit(train_x, train_y)
svr_pred = svr_model.predict(test_x)
svr_score = svr_model.score(test_x, test_y)
svr_mse = mean_squared_error(test_y, svr_pred)

print("SVR R-squared Score:", svr_score)
print("SVR Mean Squared Error:", svr_mse)

# Lasso
lasso_model = Lasso()
lasso_model.fit(train_x, train_y)
lasso_pred = lasso_model.predict(test_x)
lasso_score = lasso_model.score(test_x, test_y)
lasso_mse = mean_squared_error(test_y, lasso_pred)

print("Lasso R-squared Score:", lasso_score)
print("Lasso Mean Squared Error:", lasso_mse)

R-squared Score: 0.9101203505156658
Mean Squared Error: 8703953859.658112
SVR R-squared Score: -0.05448300013248408
SVR Mean Squared Error: 102116234671.63979
Lasso R-squared Score: 0.9101246000582262
Lasso Mean Squared Error: 8703542333.549786


/Users/wojtekmarszalek/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [71]:
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Ridge Regression
ridge_model = Ridge()
ridge_model.fit(train_x, train_y)
ridge_pred = ridge_model.predict(test_x)
ridge_score = ridge_model.score(test_x, test_y)
ridge_mse = mean_squared_error(test_y, ridge_pred)

print("Ridge Regression R-squared Score:", ridge_score)
print("Ridge Regression Mean Squared Error:", ridge_mse)

# ElasticNet Regression
elasticnet_model = ElasticNet()
elasticnet_model.fit(train_x, train_y)
elasticnet_pred = elasticnet_model.predict(test_x)
elasticnet_score = elasticnet_model.score(test_x, test_y)
elasticnet_mse = mean_squared_error(test_y, elasticnet_pred)

print("ElasticNet Regression R-squared Score:", elasticnet_score)
print("ElasticNet Regression Mean Squared Error:", elasticnet_mse)

# Decision Tree Regression
dt_model = DecisionTreeRegressor()
dt_model.fit(train_x, train_y)
dt_pred = dt_model.predict(test_x)
dt_score = dt_model.score(test_x, test_y)
dt_mse = mean_squared_error(test_y, dt_pred)

print("Decision Tree Regression R-squared Score:", dt_score)
print("Decision Tree Regression Mean Squared Error:", dt_mse)

# Random Forest Regression
rf_model = RandomForestRegressor()
rf_model.fit(train_x, train_y)
rf_pred = rf_model.predict(test_x)
rf_score = rf_model.score(test_x, test_y)
rf_mse = mean_squared_error(test_y, rf_pred)

print("Random Forest Regression R-squared Score:", rf_score)
print("Random Forest Regression Mean Squared Error:", rf_mse)

# Gradient Boosting Regression
gb_model = GradientBoostingRegressor()
gb_model.fit(train_x, train_y)
gb_pred = gb_model.predict(test_x)
gb_score = gb_model.score(test_x, test_y)
gb_mse = mean_squared_error(test_y, gb_pred)

print("Gradient Boosting Regression R-squared Score:", gb_score)
print("Gradient Boosting Regression Mean Squared Error:", gb_mse)

Ridge Regression R-squared Score: 0.9102098443397519
Ridge Regression Mean Squared Error: 8695287269.166943
ElasticNet Regression R-squared Score: 0.7909537601290451
ElasticNet Regression Mean Squared Error: 20244057879.742313
Decision Tree Regression R-squared Score: 0.9547354081509554
Decision Tree Regression Mean Squared Error: 4383427407.5469265


/var/folders/lv/5hlry00j1vl3lmxtg4gj8r6r0000gn/T/ipykernel_8212/3510769580.py:37: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(train_x, train_y)


Random Forest Regression R-squared Score: 0.9809998530511149
Random Forest Regression Mean Squared Error: 1839976049.2907114
Gradient Boosting Regression R-squared Score: 0.9828840217703614
Gradient Boosting Regression Mean Squared Error: 1657512970.15965


/Users/wojtekmarszalek/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
